In [ ]:
!pip install requests transformers newsapi-python gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import requests
import gradio as gr
from transformers import pipeline

# Set up the Hugging Face summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# News API Configuration
API_KEY = "your_api_key"
NEWS_API_URL = "https://newsapi.org/v2/top-headlines"

# Function to fetch news articles
def get_news(category, country="in"):
    params = {
        "country": country,
        "category": category,
        "apiKey": API_KEY,
        "pageSize": 5  # Fetch top 5 articles
    }
    response = requests.get(NEWS_API_URL, params=params)
    if response.status_code == 200:
        return response.json().get("articles", [])
    else:
        return []

# Function to summarize news
def summarize_article(text, max_len=120):
    summary = summarizer(text, max_length=max_len, min_length=50, do_sample=False)
    return summary[0]["summary_text"]

# Gradio UI function
def summarize_news(category):
    articles = get_news(category)
    results = []

    for article in articles:
        title = article["title"]
        source = article["source"]["name"]
        url = article["url"]
        content = article["content"]

        if content:
            summary = summarize_article(content)
            results.append(f"📰 **{title}**\n📌 Source: {source}\n🔗 [Read Full Article]({url})\n\n✍️ **Summary:** {summary}\n---")

    return "\n".join(results) if results else "No articles found. Try a different category."

# Gradio Interface
categories = ["general", "business", "entertainment", "health", "science", "sports", "technology"]

gr.Interface(
    fn=summarize_news,
    inputs=gr.Dropdown(categories, label="Choose News Category"),
    outputs="markdown",
    title="📰 AI News Summarizer",
    description="Fetches and summarizes the latest news articles using AI.",
).launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b00a64a9ec55e60c63.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
